In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:

pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [128]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import datetime as dt
from collections import Counter
from operator import itemgetter
from itertools import chain

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [8]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [9]:
import json
with open('mpd_slice_0_999.json') as json_file:
    data = json.load(json_file)

In [236]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_div(artists):
    # print('get_artist_div')
    # print(artists)
    # print(set(artists))
    # print(len(artists))
    # print(len(set(artists)))
    # print()
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_div = unique_artists / total_appearances
    return artist_div

In [135]:
MS_IN_DAY = 86400
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = {}
    dates = {}
    tracks = sp.tracks(uris)['tracks']
    none_uris = []
    for uri, track in zip(uris, tracks):
        if track is None:
            print(track)
            none_uris.append(uri)
        else:
            artists[uri] = pd.DataFrame(track['artists'])['name'].values.tolist()
            #print(track['artists'])
            #print(artists[uri])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates[uri] = ms_date / MS_IN_DAY
            
    return artists, dates, none_uris

In [120]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(playlist_uris):
    remainder = len(playlist_uris) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(playlist_uris):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    none_uris = []
    remainder = len(playlist_uris) % 50
    artists, dates = {}, {}
    if remainder > 0:
        artists, dates, none_uris_iter = get_artists_and_dates(playlist_uris[:remainder])
        none_uris += none_uris_iter
    while remainder < len(playlist_uris):
        next_artists, next_dates, none_uris_iter = get_artists_and_dates(playlist_uris[remainder:remainder+50])

        artists.update(next_artists)
        dates.update(next_dates)
        remainder += 50
        none_uris += none_uris_iter

    # for i in range(len(all_features)):
    #     print(i)
    #     print(all_features[i]['uri'])
    #     print(artists.keys()[i])
    return all_features, artists, dates, none_uris

In [112]:
# splits out uris that we have already called the api for
# also eliminates duplicate songs within a playlist
def deduplicate_uris(playlist_uris, used_tracks_by_uri):
    duplicates = np.intersect1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    new_uris = np.setdiff1d(playlist_uris, np.array(list(used_tracks_by_uri.keys())))
    return duplicates, new_uris#, count_ids

In [232]:
# make sure artists and dates get added to used_tracks_by_uri entries
# 

# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features(tracks, used_tracks_by_uri, used_artists_by_uri, used_dates_by_uri):

    tracks = pd.DataFrame(tracks)
    duplicates, new_uris = deduplicate_uris(tracks['track_uri'], used_tracks_by_uri) # deduplicate here

    all_features, artists, dates, none_uris = get_playlist_audio_features_batch(new_uris)

    # add duplicates to this playlist for aggregate metric calculation
    dup_artists = []
    dup_dates = []
    if len(duplicates) > 1:
        all_features += list(itemgetter(*duplicates)(used_tracks_by_uri))
        dup_artists = list(itemgetter(*duplicates)(used_artists_by_uri)) # artists is a dict - update it accordingly
        dup_dates = list(itemgetter(*duplicates)(used_dates_by_uri))
    elif len(duplicates) == 1:
        all_features.append(used_tracks_by_uri[duplicates[0]])
        dup_artists = used_artists_by_uri[duplicates[0]]
        dup_dates = [used_dates_by_uri[duplicates[0]]]

    all_features = pd.DataFrame(all_features)
    
    # update list of already used tracks
    for i, uri in zip(range(len(new_uris)), new_uris):
        if uri not in none_uris:
            used_tracks_by_uri[uri] = all_features.loc[all_features['uri'] == uri].to_dict('records')[0]
            used_artists_by_uri[uri] = artists[uri]
            used_dates_by_uri[uri] = dates[uri]


    # calculate aggregate features
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)

    #print(list(chain(*artists.values())))
    aggregate_features['date'] = np.mean(list(dates.values()) + dup_dates)
    aggregate_features['date_var'] = np.var(list(dates.values()) + dup_dates)
    # print(dup_artists)
    # print(list(chain(*dup_artists)))
    # print('len(dup_dates) =', len(dup_dates))
    if (len(dup_dates) > 1):
        dup_artists = list(chain(*dup_artists))
    aggregate_features['artist_div'] = get_artist_div(list(chain(*artists.values())) + dup_artists)
    return aggregate_features

In [233]:
# build the aggregate dataset!
def build_dataset(playlists):
    aggregate_dataset = []
    used_tracks_by_uri = {}
    used_artists_by_uri = {}
    used_dates_by_uri = {}
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features(playlist['tracks'], used_tracks_by_uri, used_artists_by_uri, used_dates_by_uri))
    return aggregate_dataset

In [239]:
#[497:498] has a uri that returns None from the API in it, uri 19 in the list
# somewhere after it has a mean of empty slice situation happening
dataset = build_dataset(data['playlists'][:99])
print(dataset[-1])


KeyboardInterrupt: 

In [212]:
#print(pd.DataFrame(data['playlists'][98:99])['tracks'].to_list()[0])
print_list = pd.DataFrame(pd.DataFrame(data['playlists'][98:99])['tracks'].to_list()[0])['artist_name'].to_list()
print(print_list)
print(set(print_list))
print(len(print_list))
print(len(set(print_list)))

['Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews Band', 'Dave Matthews

In [180]:
dataset.to_csv(r'csvdata_nodup_10.csv')
print(dataset)

    danceability  danceability_var    energy  energy_var   loudness  \
0       0.668804          0.020467  0.777961    0.016150  -4.908451   
1       0.492382          0.018813  0.695923    0.043638  -8.107974   
2       0.671063          0.011819  0.692953    0.030439  -4.875594   
3       0.514429          0.030802  0.620902    0.058971  -9.618754   
4       0.576235          0.028260  0.650418    0.040555  -7.634529   
..           ...               ...       ...         ...        ...   
95      0.742185          0.016337  0.616226    0.021138  -6.659560   
96      0.507053          0.024533  0.471737    0.044413  -7.775105   
97      0.522312          0.023039  0.599708    0.050582  -8.448922   
98      0.482805          0.013766  0.676156    0.049587  -8.001727   
99      0.578167          0.013402  0.596190    0.044780 -10.800762   

    loudness_var  speechiness  speechiness_var  acousticness  \
0       1.740075     0.104486         0.007470      0.085282   
1       6.896997   

In [179]:
times = [1, 10, 100, 1000]
for i in times:
    t = timeit.Timer('build_dataset(data[\'playlists\'][:i])', 'from __main__ import build_dataset, data, i')
    print('Time: ' + str(t.timeit(1)))

Time: 0.49905658300031064
Time: 4.488395583999591
Time: 49.742303917000754
None
None
Time: 418.37758374999976


In [ ]:
#207 dates mean of empty slice